1. LLM獲取文字
2. 判斷人物、說話語氣
3. TTS轉語音

In [1]:
import gc
import re
import json
import pandas as pd
from pydub import AudioSegment

from TTS.api import TTS
from gliner import GLiNER
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

from utils import *

device = "cuda" if torch.cuda.is_available() else "cpu"

C:\Users\USER\anaconda3\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


ModuleNotFoundError: No module named 'TTS'

In [2]:
gn = GLiNER.from_pretrained("urchade/gliner_mediumv2.1").to(device)
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)
classifier = pipeline("sentiment-analysis", model="michellejieli/emotion_text_classifier",  device=device)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model.safetensors:  11%|#         | 83.9M/781M [00:00<?, ?B/s]

pytorch_model.bin:  13%|#3        | 105M/781M [00:00<?, ?B/s]

C:\Users\USER\anaconda3\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub\models--urchade--gliner_mediumv2.1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

C:\Users\USER\anaconda3\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub\models--microsoft--deberta-v3-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

C:\Users\USER\anaconda3\lib\site-packages\transformers\convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [13]:
result = main()

df = pd.DataFrame(result)
df['emotion'] = None
df['char'] = None

labels = ["Male", "Female"]

# 獲取情緒和角色身分
for c in range(df.shape[0]):
    sentence_with_name = f"{df['speaker'][c]} : {df['content'][c]}"
    sentence_only = df['content'][c]
    
    classify_result = classifier(sentence_only)
    gn_result = gn.predict_entities(sentence, labels, threshold=0.5)
    
    df.loc[c, 'emotion'] = classify_result[0]['label'] 
    df.loc[c, 'char'] = gn_result[0]['label']
    
# 文字到語音
for d in range(df.shape[0]):
    
    tts.tts_to_file(
        text="Hello world!",
        speaker = df['char'][d]
        language="en"
        speaker_wav="03-01-01-01-01-01-06.wav",
        language="en",
        emotion = df['emotion'][d]
        file_path=f"output{d}.wav")

In [ ]:
wav_files = [f"Output/sentence_{i}.wav" for i in range(1, 5)]

combined_audio = AudioSegment.from_wav(wav_files[0])

# 逐步將剩下的音頻文件合併到第一個音頻文件中
for wav_file in wav_files[1:]:
    next_audio = AudioSegment.from_wav(wav_file)
    combined_audio += next_audio

# 將合併後的音頻文件保存
combined_audio.export("Output/combined_audio.wav", format="wav")